In [1]:
using DataFrames, CSV, StringEncodings, Dictionaries, Dates, StatsBase, HTTP, JSON

In [2]:
datapath = joinpath("..", "Data")

"../Data"

## Read in the important raw files, we'll format this into a better formatted dataset after

In [3]:
resultsdf = DataFrame(CSV.File(open(joinpath(datapath, "mlb-2025-asplayed.csv"), enc"ISO-8859-1")));
pitchingdf = DataFrame(CSV.File(joinpath(datapath, "Pitching_data.csv")));

In [4]:
# We need to reconcile the naming conventions - resultsdf has typed out teamnames while pitchingdf has shortforms and pks

teammap = DataFrame(CSV.File(joinpath(datapath, "TeamNameMap.csv")))
applymap = Dictionary(teammap.verbose, teammap.code)

resultsdf.HomeID = getindices(applymap, resultsdf.Home)
resultsdf.AwayID = getindices(applymap, resultsdf.Away);

In [20]:
pitchingdf.HomeID .= 0
pitchingdf.AwayID .= 0
pitchingdf.HomeRuns .= 0
pitchingdf.AwayRuns .= 0

for pk = unique(pitchingdf.game_pk)
    # Get the game 
    r = HTTP.request("GET", "https://baseballsavant.mlb.com/gf?game_pk=$pk")
    gameresult = JSON.parse(String(r.body))

    pitchingdf[pitchingdf.game_pk .== pk, :HomeID] .= gameresult["scoreboard"]["teams"]["home"]["teamCode"]
    pitchingdf[pitchingdf.game_pk .== pk, :AwayID] .= gameresult["scoreboard"]["teams"]["away"]["teamCode"]
    pitchingdf[pitchingdf.game_pk .== pk, :HomeRuns] .= gameresult["scoreboard"]["linescore"]["teams"]["home"]["runs"]
    pitchingdf[pitchingdf.game_pk .== pk, :AwayRuns] .= gameresult["scoreboard"]["linescore"]["teams"]["away"]["runs"]
end

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Int64
The function `convert` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  convert(::Type{T}, !Matched::EzXML.NodeType) where T<:Integer
   @ EzXML /Users/iai/builds/e7x1Q22r/0/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/EzXML/DL8na/src/node.jl:36
  convert(::Type{T}, !Matched::EzXML.ReaderType) where T<:Integer
   @ EzXML /Users/iai/builds/e7x1Q22r/0/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/EzXML/DL8na/src/streamreader.jl:59
  convert(::Type{T}, !Matched::SentinelArrays.ChainedVectorIndex) where T<:Union{Signed, Unsigned}
   @ SentinelArrays /Users/iai/builds/e7x1Q22r/0/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/SentinelArrays/V85ev/src/chainedvector.jl:215
  ...


In [5]:
# To match the game primary key let's setup a matching column that is of the format yymmddHHH where H is the
# home team identifier

tojoin = pitchingdf[:, [:player_id, :game_date, :game_pk]]
tojoin.joincol = string.(Dates.format.(tojoin.game_date, dateformat"yymmdd"), tojoin.player_id)

resultsdf.joincol = string.(Dates.format.(resultsdf.Date, dateformat"yymmdd"), resultsdf.HomeID)

dblheaders = [k for (k, v) in countmap(tojoin.joincol) if v > 1]

tojoin = filter(row -> !(row.joincol in dblheaders), tojoin)
tojoin = tojoin[:, [:game_pk, :joincol]]

resultsdf = leftjoin(resultsdf, tojoin, on = :joincol)

CSV.write("formanualdblheaders.csv", filter(row -> ismissing(row.game_pk), resultsdf))

"formanualdblheaders.csv"

In [13]:
pk = tojoin[1, :game_pk]
print(pk)

r = HTTP.request("GET", "https://baseballsavant.mlb.com/gf?game_pk=$pk")
println(r.status)
gameresult = JSON.parse(String(r.body))
k = keys(gameresult)
println(gameresult["scoreboard"]["linescore"]["teams"])
print(keys(gameresult["scoreboard"]["teams"]["home"]))

778501200
Dict{String, Any}("home" => Dict{String, Any}("errors" => 1, "hits" => 10, "leftOnBase" => 8, "runs" => 3), "away" => Dict{String, Any}("errors" => 0, "hits" => 21, "leftOnBase" => 15, "runs" => 18))
["teamName", "active", "clubName", "link", "springVenue", "name", "id", "venue", "season", "firstYearOfPlay", "allStarStatus", "fileCode", "springLeague", "sport", "shortName", "franchiseName", "record", "abbreviation", "locationName", "league", "division", "teamCode"]